Added Seasons and got better validation than training. Might remove
Best estimator at max_depth = 20 and n_estimator=300. Better val than training.
Test baseline data with best estimator and see results

In [1]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score, accuracy_score, balanced_accuracy_score
from scipy.stats import randint
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.pipeline import make_pipeline

In [2]:
import pandas as pd
import numpy as np
import random

pd.set_option('display.max_columns', 100)

# Open, Read, and Save to data frame
X_train = 'train_features.csv'
X_test = 'test_features.csv'
y_train = 'train_labels.csv'

X_train = pd.read_csv(X_train)
X_test = pd.read_csv(X_test)
y_train = pd.read_csv(y_train)

# Will need to split X_train and y_train to training and validation
X_train.head()

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109,True,GeoData Consultants Ltd,VWC,Roman,False,1999,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280,NaN,GeoData Consultants Ltd,Other,NaN,True,2010,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58,True,GeoData Consultants Ltd,VWC,NaN,True,1986,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,0,True,GeoData Consultants Ltd,NaN,NaN,True,0,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe


In [ ]:
X_train[X_train['district_code']==1]['water_quality'].mode()

## Feature Engineer

In [ ]:
# Function to create seasons
# def season(df=X_train, month=3, season='spring'):
#     if month != 12:
#         df['season'] = df['season'].replace(month,season)
#         df['season'] = df['season'].replace(month+1,season)
#         df['season'] = df['season'].replace(month+2,season)
#         return df['season']
#     else:
#         df['season'] = df['season'].replace(month,season)
#         df['season'] = df['season'].replace(1,season)
#         df['season'] = df['season'].replace(2,season)
#         return df['season']

In [3]:
#X_train['long_and_lat'] = list(zip(X_train['longitude'], X_train['latitude']))
#X_test['long_and_lat'] = list(zip(X_test['longitude'], X_test['latitude']))

X_train['is_construct'] = X_train['construction_year'] != 0
X_test['is_construct'] = X_test['construction_year'] != 0

#X_train['dirty_quality'] = X_train['water_quality'] != 'soft' 
#X_test['dirty_quality'] = X_test['water_quality'] != 'soft'

X_train['Water_per_person'] = X_train['amount_tsh']/X_train['population']
X_train['Water_per_person'] = X_train['Water_per_person'].replace([np.inf, -np.inf], np.nan)
X_train['Water_per_person'] = X_train['Water_per_person'].replace(np.nan, 0)

X_test['Water_per_person'] = X_test['amount_tsh']/X_test['population']
X_test['Water_per_person'] = X_test['Water_per_person'].replace([np.inf, -np.inf], np.nan)
X_test['Water_per_person'] = X_test['Water_per_person'].replace(np.nan, 0)

#X_train['Water_per_person'] = X_train['Water_per_person'].replace(0, round(pd.concat([X_train,X_test])['Water_per_person'].median()))
#X_test['Water_per_person'] = X_test['Water_per_person'].replace(0, round(pd.concat([X_train,X_test])['Water_per_person'].median()))


#X_train['has_water'] = X_train['amount_tsh'] != 0
#X_test['has_water'] = X_test['amount_tsh'] != 0

X_train['age'] = 2018 - X_train['construction_year']
X_train['age'] = X_train['age'].replace(2018,0)
X_test['age'] =  2018 - X_test['construction_year']
X_test['age'] = X_test['age'].replace(2018,0)

X_train['date_recorded'] = pd.to_datetime(X_train['date_recorded'], infer_datetime_format=True)
X_train['week'] = X_train['date_recorded'].apply(lambda x: x.week)
X_train['month'] = X_train['date_recorded'].apply(lambda x: x.month)

# Create season from month
# X_train['season'] = X_train['month']
# X_train['season'] = season(X_train, 3, 'spring')
# X_train['season'] = season(X_train, 6, 'summer')
# X_train['season'] = season(X_train, 9, 'fall')
# X_train['season'] = season(X_train, 12, 'winter')
# X_train['Spring'] = (X_train['month']== 3) | (X_train['month']== 4) | (X_train['month']== 5)
# X_train['Summer'] = (X_train['month']== 6) | (X_train['month']== 7) | (X_train['month']== 8)
# X_train['Fall'] = (X_train['month']== 9) | (X_train['month']== 10) | (X_train['month']== 11)
# X_train['Winter'] = (X_train['month']== 12) | (X_train['month']== 1) | (X_train['month']== 2)

# Change back to string or else RandomForestClassifier can't process
X_train['date_recorded']= X_train['date_recorded'].astype(str)

X_test['date_recorded'] = pd.to_datetime(X_test['date_recorded'], infer_datetime_format=True)
X_test['week'] = X_test['date_recorded'].apply(lambda x: x.week)
X_test['month'] = X_test['date_recorded'].apply(lambda x: x.month)

# X_test['season'] = X_test['month']
# X_test['season'] = season(X_test, 3, 'spring')
# X_test['season'] = season(X_test, 6, 'summer')
# X_test['season'] = season(X_test, 9, 'fall')
# X_test['season'] = season(X_test, 12, 'winter')

# X_test['Spring'] = (X_test['month']== 3) | (X_test['month']== 4) | (X_test['month']== 5)
# X_test['Summer'] = (X_test['month']== 6) | (X_test['month']== 7) | (X_test['month']== 8)
# X_test['Fall'] = (X_test['month']== 9) | (X_test['month']== 10) | (X_test['month']== 11)
# X_test['Winter'] = (X_test['month']== 12) | (X_test['month']== 1) | (X_test['month']== 2)

# Change back to string or else RandomForestClassifier can't process
X_test['date_recorded'] = X_test['date_recorded'].astype(str)

In [ ]:
X_train['water_quality'].value_counts()

In [ ]:
X_train['quantity'].value_counts()

In [ ]:
X_train['quality_group'].value_counts()

In [ ]:
X_train.isnull().sum()

In [ ]:
X_train.head()

In [ ]:

X_train['season'].value_counts()

## Fill NaN

In [4]:
# def fill_mode(df, col):
#     return df[col].fillna(df[col].mode()[0])

# X_train['funder'] = fill_mode(X_train, 'funder')
# X_test['funder'] = fill_mode(X_test, 'funder')

# X_train['installer'] = fill_mode(X_train, 'installer')
# X_test['installer'] = fill_mode(X_test, 'installer')

# X_train['scheme_management'] = fill_mode(X_train, 'scheme_management')
# X_test['scheme_management'] = fill_mode(X_test, 'scheme_management')

# X_train['subvillage'] = fill_mode(X_train, 'subvillage')
# X_test['subvillage'] = fill_mode(X_test, 'subvillage')

# Forward fill
X_train['public_meeting'] = X_train['public_meeting'].fillna(lambda x: random.choice(X_train[X_train['public_meeting'] != np.nan])['public_meeting'])
X_test['public_meeting'] = X_test['public_meeting'].fillna(lambda x: random.choice(X_test[X_test['public_meeting'] != np.nan])['public_meeting'])

X_train['permit'] = X_train['permit'].fillna(lambda x: random.choice(X_train[X_train['permit'] != np.nan])['permit'])
X_test['permit'] = X_test['permit'].fillna(lambda x: random.choice(X_test[X_test['permit'] != np.nan])['permit'])

In [ ]:
X_train.isna().sum()

In [ ]:
X_train['quantity'].value_counts()

In [ ]:
# Find mean/median/mode of X_train and X_test 
concat = pd.concat([X_train,X_test])

## Replace 0 with the mean - Most improved

In [5]:
#Replace 0 with mean - Check continuous_col and drop_cols

#X_train['amount_tsh'] = X_train['amount_tsh'].replace(0, round(pd.concat([X_train,X_test])['amount_tsh'].mean()))
#X_test['amount_tsh'] = X_test['amount_tsh'].replace(0, round(pd.concat([X_train,X_test])['amount_tsh'].mean()))

X_train['age'] = X_train['age'].replace(0, round(pd.concat([X_train,X_test])['age'].mean()))
X_test['age'] = X_test['age'].replace(0, round(pd.concat([X_train,X_test])['age'].mean()))

X_train['gps_height'] = X_train['gps_height'].replace(0, round(pd.concat([X_train,X_test])['gps_height'].mean()))
X_test['gps_height'] = X_test['gps_height'].replace(0, round(pd.concat([X_train,X_test])['gps_height'].mean()))

## Replace 0 with the median

In [ ]:
#Replace 0 with median - Added 'amount_tsh' to continous_col. drop_cols # out

#Pop median did not score better
#X_train['population'] = X_train['population'].replace(0, X_train['population'].median())
#X_test['population'] = X_test['population'].replace(0, X_test['population'].median())

# X_train['amount_tsh'] = X_train['amount_tsh'].replace(0, X_train['amount_tsh'].median())
# X_test['amount_tsh'] = X_test['amount_tsh'].replace(0, X_test['amount_tsh'].median())

# X_train['age'] = X_train['age'].replace(0, X_train['age'].median())
# X_test['age'] = X_test['age'].replace(0, X_test['age'].median())

# X_train['gps_height'] = X_train['gps_height'].replace(0, X_train['gps_height'].median())
# X_test['gps_height'] = X_test['gps_height'].replace(0, X_test['gps_height'].median())

## Replace with mode

In [ ]:
#X_train['funder'] = X_train['funder'].fillna(pd.concat([X_train,X_test])['funder'].mode()[0])
#X_test['funder'] = X_test['funder'].fillna(pd.concat([X_train,X_test])['funder'].mode()[0])

#X_train['installer'] = X_train['installer'].fillna(pd.concat([X_train,X_test])['funder'].mode()[0])
#X_test['installer'] = X_test['installer'].fillna(pd.concat([X_train,X_test])['funder'].mode()[0])

#X_train['scheme_management'] = X_train['scheme_management'].fillna(pd.concat([X_train,X_test])['scheme_management'].mode()[0])
#X_test['scheme_management'] = X_test['scheme_management'].fillna(pd.concat([X_train,X_test])['scheme_management'].mode()[0])

In [ ]:
X_train.head()

In [ ]:
pd.concat([X_train,X_test])['funder'].mode()[0]

In [ ]:
X_train.isna().sum()

## Drop Features

In [ ]:
#Baseline drop - Don't add features. Add Fill Nan. NO scale
# drop_cols = ['recorded_by','management','id','source_class','wpt_name']
# X_train = X_train.drop(columns=drop_cols)
# X_test = X_test.drop(columns=drop_cols)

In [ ]:
# Merge
# X_train = X_train.merge(y_train)

In [ ]:
# Remove cols and rows with NaN
# feature_remove = ['funder','installer','subvillage','public_meeting','scheme_name']
# #observation_remove = ['scheme_management','permit']

# X_train = X_train.drop(columns=feature_remove)#.dropna(subset=observation_remove)
# X_test = X_test.drop(columns=feature_remove)#.dropna(subset=observation_remove)


In [ ]:
# Seperate X_train and y_train from X_train
# drop_cols= ['id']

# y_train = X_train.loc[:,['id','status_group']]
# X_train = X_train.drop(columns='status_group')
##X_test = X_test.drop(columns=drop_cols)

In [6]:
drop_cols = ['quantity_group','construction_year','recorded_by','id','num_private','amount_tsh', 'wpt_name','subvillage','source_class','management_group']
#drop_cols = ['amount_tsh', 'wpt_name','subvillage'] add for the mean
#drop_cols = ['wpt_name', 'management_group','payment'] add for the median
# quality_group and water_quality are similar. when dropping quality_group it did slightly worst
X_train = X_train.drop(columns=drop_cols)
X_test = X_test.drop(columns=drop_cols)

In [ ]:
column_name = X_train.columns.tolist()
#########################################################

In [7]:
# Drop id
y_train = y_train.drop(columns='id')

In [8]:
# look at shape
X_train.shape, y_train.shape, X_test.shape

((59400, 35), (59400, 1), (14358, 35))

In [9]:
# Split X_train to training and validation
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=42, test_size=.2)

In [10]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((47520, 35), (47520, 1), (11880, 35), (11880, 1))

In [ ]:
# Make Pipeline
# pipe = make_pipeline(
#     ce.OrdinalEncoder(),
#     StandardScaler()
# )

In [ ]:
X_train.head()

In [11]:
# Encoder Categorical and fit_transform
encoder = ce.OrdinalEncoder()
X_train = encoder.fit_transform(X_train)

In [12]:
# Scale
continuous_col = ['population','gps_height','age','week','month']
scaled = MinMaxScaler()
X_train[continuous_col] = scaled.fit_transform(X_train[continuous_col])

C:\Users\Dustin\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [14]:
# Define parameters for the model
param_distributions = {
    'n_estimators': [300,325,350,375,400],#randint(50,300)
    'max_depth': [15,20,25]#randint(2,20)
}

# Define the model or input inside RSCV
model = RandomForestClassifier(n_jobs=-1, random_state=42)

# RSCV
search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    scoring='accuracy',
    n_iter=15,
    n_jobs=-1,
    cv=5,
    verbose=10,
    return_train_score=True,
    random_state=42
)

# Fit to training data
search.fit(X_train, y_train)

# Best Score? scoring=''
print('Training Accuracy Score:',search.best_score_)


Fitting 5 folds for each of 15 candidates, totalling 75 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:   24.6s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:   51.9s
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:   55.2s
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done  45 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done  68 out of  75 | elapsed:  4.7min remaining:   28.7s
[Parallel(n_jobs=-1)]: Done  75 out of  75 | elapsed:  4.9min finished
C:\Users\Dustin\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


Training Accuracy Score: 0.8095749158249158


In [15]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
9,28.824903,1.537027,2.176379,0.523077,400,20,"{'n_estimators': 400, 'max_depth': 20}",0.811487,0.809153,0.807114,0.812270,0.807850,0.809575,0.002007,1,0.962224,0.963593,0.962596,0.963832,0.964174,0.963284,0.000746
8,26.845000,1.618977,4.320245,1.052662,375,20,"{'n_estimators': 375, 'max_depth': 20}",0.811382,0.809258,0.807114,0.811849,0.807850,0.809491,0.001873,2,0.962172,0.963488,0.962701,0.963963,0.964042,0.963273,0.000729
7,26.512887,0.230575,2.061486,0.435123,350,20,"{'n_estimators': 350, 'max_depth': 20}",0.811382,0.809574,0.806693,0.812270,0.807429,0.809470,0.002163,3,0.962303,0.963488,0.962648,0.964042,0.964542,0.963405,0.000836
6,22.986719,0.991724,3.901564,1.693824,325,20,"{'n_estimators': 325, 'max_depth': 20}",0.811487,0.808943,0.806798,0.811638,0.807745,0.809322,0.001952,4,0.962435,0.963593,0.962701,0.964121,0.964411,0.963452,0.000773
5,22.310128,1.454475,3.272048,1.034505,300,20,"{'n_estimators': 300, 'max_depth': 20}",0.810856,0.809574,0.806903,0.810797,0.808166,0.809259,0.001534,5,0.962382,0.963804,0.962911,0.964069,0.964069,0.963447,0.000681
14,21.759599,7.864871,1.135367,0.978124,400,25,"{'n_estimators': 400, 'max_depth': 25}",0.807595,0.809363,0.807008,0.810060,0.803430,0.807492,0.002317,6,0.994634,0.994792,0.994581,0.994713,0.994739,0.994692,0.000075
13,28.921446,1.357825,3.778893,0.887514,375,25,"{'n_estimators': 375, 'max_depth': 25}",0.807911,0.809258,0.806482,0.809534,0.802904,0.807218,0.002415,7,0.994712,0.994870,0.994529,0.994792,0.994792,0.994739,0.000116
12,27.316139,0.668808,2.739473,1.271919,350,25,"{'n_estimators': 350, 'max_depth': 25}",0.807174,0.808522,0.806693,0.809113,0.802904,0.806881,0.002173,8,0.994739,0.994870,0.994660,0.994765,0.994765,0.994760,0.000067
11,24.334114,1.054404,3.908745,0.660359,325,25,"{'n_estimators': 325, 'max_depth': 25}",0.806228,0.807891,0.805851,0.809534,0.803010,0.806503,0.002182,9,0.994607,0.994870,0.994450,0.994844,0.994634,0.994681,0.000157
10,25.597336,0.313021,2.213679,0.760838,300,25,"{'n_estimators': 300, 'max_depth': 25}",0.805912,0.807575,0.805430,0.808902,0.803010,0.806166,0.002002,10,0.994712,0.994739,0.994450,0.994608,0.994634,0.994629,0.000102


In [16]:
# Fit X_val
best = search.best_estimator_
X_val = encoder.transform(X_val)
print(best)
# Partial scale x_val
X_val[continuous_col] = scaled.fit_transform(X_val[continuous_col])

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=20, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=400, n_jobs=-1,
            oob_score=False, random_state=42, verbose=0, warm_start=False)


C:\Users\Dustin\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [17]:
# Test Validation
best = search.best_estimator_
y_pred = best.predict(X_val)
print('Validation Set Accuracy Score:', accuracy_score(y_val, y_pred))

Validation Set Accuracy Score: 0.8137205387205387


In [18]:
import eli5
from eli5.sklearn import PermutationImportance

# Instantiate. prefit means cv already done
permuter = PermutationImportance(best, scoring='accuracy', cv='prefit', n_iter=3,
                                 random_state=42)
# Fit to test- Not sure if we use .values or without
permuter.fit(X_val, y_val)

# Show - Try without .tolist() if not then use it
feature_names = X_val.columns.tolist()
eli5.show_weights(permuter, top=None, feature_names=feature_names)

Weight,Feature
0.0994 ± 0.0075,quantity
0.0144 ± 0.0023,age
0.0117 ± 0.0017,waterpoint_type
0.0109 ± 0.0023,extraction_type_class
0.0097 ± 0.0023,Water_per_person
0.0082 ± 0.0034,longitude
0.0058 ± 0.0023,latitude
0.0042 ± 0.0007,population
0.0032 ± 0.0010,scheme_name
0.0027 ± 0.0002,lga


## RETEST

In [ ]:
# Save features with number greater than 0
mask = permuter.feature_importances_ > 0
feature = X_train.columns[mask]
X_train = X_train[feature]

# Define parameters for the model
param_distributions = {
    'n_estimators': [175,200,225,250],#randint(50,300),
    'max_depth': [20,25,30]#randint(2,20)
}

# Define the model or input inside RSCV
model = RandomForestClassifier(n_jobs=-1, random_state=42)

# RSCV
search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_distributions,
    scoring='accuracy',
    n_iter=12,
    n_jobs=-1,
    cv=5,
    verbose=10,
    return_train_score=True,
    random_state=42
)

# Fit to training data
search.fit(X_train, y_train)

# Best Score? scoring=''
print('Training Accuracy Score:',search.best_score_)


In [ ]:
# Test Validation
best = search.best_estimator_
X_val = X_val[feature]
y_pred = best.predict(X_val)
print('Validation Set Accuracy Score:', accuracy_score(y_val, y_pred))

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

# Instantiate. prefit means cv already done
permuter = PermutationImportance(best, scoring='accuracy', cv='prefit', n_iter=3,
                                 random_state=42)
# Fit to test- Not sure if we use .values or without
permuter.fit(X_val, y_val)

# Show - Try without .tolist() if not then use it
feature_names = X_val.columns.tolist()
eli5.show_weights(permuter, top=None, feature_names=feature_names)

In [ ]:
import matplotlib.pyplot as plt
# Length of columns
n = len(X_train.columns)

# Feature importances from best_estimator_
importances = pd.Series(best.feature_importances_, X_train.columns)

# Top feature sorted by descending value. Sliced from bottom to top
top_n = importances.sort_values()[-n:]

# Plot
plt.figure(figsize=(5,15))
top_n.plot.barh()
plt.show()

In [ ]:
# No Retest

# Test X_Test
best = search.best_estimator_
X_test = encoder.transform(X_test)

# Partial scale x_test
X_test[continuous_col] = scaled.fit_transform(X_test[continuous_col])
y_pred_test = best.predict(X_test)

In [ ]:
# With Retest

# Check previous feature importances and add/delete accordingly
continuous_col = ['population','gps_height','age','week','month']

# Test X_Test
best = search.best_estimator_
X_test = encoder.transform(X_test[feature])

# Partial scale x_test
X_test[continuous_col] = scaled.fit_transform(X_test[continuous_col])
y_pred_test = best.predict(X_test)

In [ ]:
best

In [ ]:
### SUBMISSION ###

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred_test
submission.to_csv('First-submission.csv', index=False)

In [ ]:
search.best_estimator_

In [ ]:
y_pred_test.shape

In [ ]:
X_train.head()

In [ ]:
X_test.shape

In [ ]:
y_train['status_group'].value_counts(normalize=True)

In [ ]:
pd.Series(y_pred_test).value_counts(normalize=True)